In [13]:
import sys
import os
sys.path.append(os.path.abspath("../"))  # agrega la raíz del proyecto al path


In [18]:
from pathlib import Path
import pandas as pd

# Define la raíz del proyecto de forma dinámica (sube al nivel del proyecto)
root_dir = Path().resolve().parent  # ← asume que estás en /notebooks

# Construye la ruta relativa a partir de la raíz
input_path = root_dir / "data" / "interim" / "datos_llamadas.csv"

# Cargar el archivo
df_raw = pd.read_csv(input_path)
print(df_raw.columns)


Index(['conversation_id', 'cliente', 'idioma', 'queue_name', 'weekday',
       'date_time', 'offered', 'answered', 'service_level_%_30_sec',
       '%_abandon', 'aht', 'talk_(avg)', 'hold_(avg)', 'acw'],
      dtype='object')


In [14]:
from src.etl.transform_calls import procesar_llamadas

df = procesar_llamadas()
df.head()


✅ Dataset procesado guardado en C:\Users\gcasc\proyectos\proyecto_final\data\processed\llamadas_diarias.csv


,date,idioma,y,dayofweek,is_month_end,lag_1,lag_2,lag_3,lag_4,lag_5
46,2024-03-08,DE,21,4,0,39.0,32.0,34.0,16.0,38.0
56,2024-03-11,DE,35,0,0,21.0,39.0,32.0,34.0,16.0
67,2024-03-12,DE,49,1,0,35.0,21.0,39.0,32.0,34.0
76,2024-03-13,DE,49,2,0,49.0,35.0,21.0,39.0,32.0
84,2024-03-14,DE,39,3,0,49.0,49.0,35.0,21.0,39.0


In [19]:
from src.etl.transform_calls import procesar_llamadas

df = procesar_llamadas()
df.columns


✅ Dataset procesado guardado en C:\Users\gcasc\proyectos\proyecto_final\data\processed\llamadas_diarias.csv


Index(['date', 'idioma', 'y', 'dayofweek', 'is_month_end', 'lag_1', 'lag_2',
       'lag_3', 'lag_4', 'lag_5'],
      dtype='object')

In [17]:
df_raw = pd.read_csv("C:/Users/gcasc/proyectos/proyecto_final/data/interim/datos_llamadas.csv")
print(df_raw.columns)


Index(['conversation_id', 'cliente', 'idioma', 'queue_name', 'weekday',
       'date_time', 'offered', 'answered', 'service_level_%_30_sec',
       '%_abandon', 'aht', 'talk_(avg)', 'hold_(avg)', 'acw'],
      dtype='object')


In [21]:
from src.etl.transform_calls import procesar_llamadas

df = procesar_llamadas()
print(df.columns)


✅ Dataset procesado guardado en C:\Users\gcasc\proyectos\proyecto_final\data\processed\llamadas_diarias.csv
Index(['date', 'idioma', 'y', 'dayofweek', 'is_month_end', 'lag_1', 'lag_2',
       'lag_3', 'lag_4', 'lag_5'],
      dtype='object')


In [24]:
import pandas as pd
import os
from pathlib import Path

def procesar_llamadas():
    # Asume que el notebook está en /notebooks y sube a raíz del proyecto
    root_dir = Path().resolve().parent

    input_path = root_dir / "data" / "interim" / "datos_llamadas.csv"
    output_path = root_dir / "data" / "processed" / "llamadas_diarias.csv"

    df = pd.read_csv(input_path)
    df.columns = [c.lower().strip().replace(" ", "_").replace("/", "_") for c in df.columns]

    print("🔍 Columnas después de renombrar:")
    print(df.columns)

    if "cliente" not in df.columns:
        raise ValueError("❌ La columna 'cliente' no se encuentra en el DataFrame.")

    df["date"] = pd.to_datetime(df["date_time"]).dt.date

    print("🔁 Mostrando ejemplo antes de agrupar:")
    print(df[["date", "cliente", "idioma", "offered"]].head())

    df_grouped = (
        df.groupby(["date", "cliente", "idioma"])
        .agg({"offered": "sum"})
        .reset_index()
        .rename(columns={"offered": "y"})
    )

    print("✅ Después del groupby:")
    print(df_grouped.columns)

    df_grouped["date"] = pd.to_datetime(df_grouped["date"])
    df_grouped["dayofweek"] = df_grouped["date"].dt.dayofweek
    df_grouped["is_month_end"] = df_grouped["date"].dt.is_month_end.astype(int)

    df_grouped = df_grouped.sort_values(["cliente", "idioma", "date"])
    for lag in [1, 2, 3, 4, 5]:
        df_grouped[f"lag_{lag}"] = df_grouped.groupby(["cliente", "idioma"])["y"].shift(lag)

    df_grouped.dropna(inplace=True)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    df_grouped.to_csv(output_path, index=False)

    print(f"✅ Dataset procesado guardado en {output_path}")
    print("📦 Columnas finales:", df_grouped.columns.tolist())

    return df_grouped


In [25]:
df = procesar_llamadas()


🔍 Columnas después de renombrar:
Index(['conversation_id', 'cliente', 'idioma', 'queue_name', 'weekday',
       'date_time', 'offered', 'answered', 'service_level_%_30_sec',
       '%_abandon', 'aht', 'talk_(avg)', 'hold_(avg)', 'acw'],
      dtype='object')
🔁 Mostrando ejemplo antes de agrupar:
         date   cliente idioma  offered
0  2025-04-04      AMEX     ES        1
1  2025-04-01      AMEX     EN        1
2  2024-12-05    AIRBNB     ES        1
3  2025-02-03      AMEX     EN        1
4  2025-03-26  COLUMBUS     IT        1
✅ Después del groupby:
Index(['date', 'cliente', 'idioma', 'y'], dtype='object')
✅ Dataset procesado guardado en C:\Users\gcasc\proyectos\proyecto_final\data\processed\llamadas_diarias.csv
📦 Columnas finales: ['date', 'cliente', 'idioma', 'y', 'dayofweek', 'is_month_end', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']
